# `Nuqleon.Time`

Provides a set of abstractions over notions of time, including clocks and stopwatches.

## Reference the library

### Option 1 - Use a local build

If you have built the library locally, run the following cell to load the latest build.

In [1]:
#r "bin/Debug/net50/Nuqleon.Time.dll"

### Option 2 - Use NuGet packages

If you want to use the latest published package from NuGet, run the following cell.

In [1]:
#r "nuget:Nuqleon.Time,*-*"

## (Optional) Attach a debugger

If you'd like to step through the source code of the library while running samples, run the following cell, and follow instructions to start a debugger (e.g. Visual Studio). Navigate to the source code of the library to set breakpoints.

In [1]:
System.Diagnostics.Debugger.Launch();

## Stopwatches and clocks

This library provides abstractions over stopwatches and clocks:

* Stopwatches measure elapsed (relative) time.
* Clocks provide absolute time.

In the sample below, we show the use of an `IClock` implementation using virtual time in order to build an `IStopwatch`.

First, let's create a `VirtualTimeClock` that starts at an initial time of `42` ticks.

In [1]:
using System.Time;

var virtTime = new VirtualTimeClock(initialTime: 42L);

Next, we go ahead and create an `IStopwatchFactory` that can be used to create `IStopwatch` instances which are a generalization of `System.Diagnostics.Stopwatch`.

In [1]:
IClock clock = virtTime;
IStopwatchFactory factory = StopwatchFactory.FromClock(clock);

Given the factory, we can create new `IStopwatch` instances in ways similar to using `new Stopwatch()` or `Stopwatch.StartNew()`. In this case, we use the latter approach and observe that the stopwatch did not record any elapsed time.

In [1]:
IStopwatch sw = factory.StartNew();

Console.WriteLine(sw.ElapsedTicks);

0


One we advance the virtual time clock, the stopwatch reflects the time elapsed.

In [1]:
virtTime.Now = 43L;

Console.WriteLine(sw.ElapsedTicks);

1


If we `Stop` the stopwatch, elapsed time does no longer accumulate.

In [1]:
sw.Stop();

virtTime.Now = 45L;

Console.WriteLine(sw.ElapsedTicks);

1


Once we resume the stopwatch using `Start`, elapsed time accumulates again.

In [1]:
sw.Start();

virtTime.Now = 48L;

Console.WriteLine(sw.ElapsedTicks);

4


Finally, we can also create `IStopwatch` instances that are backed by `System.Diagnostics.Stopwatch`, as shown below.

In [1]:
IStopwatch sw = StopwatchFactory.Diagnostics.StartNew();

Console.WriteLine(sw.ElapsedTicks);

639


Using the abstractions provided in this library, one can parameterize libraries on notions of time, while keeping them testable. For example, `Nuqleon.Memory` uses `IStopwatch[Factory]` instances to measure elapsed time for cache operations. Testing of this library is deterministic by using stopwatches based on virtual clocks.

Clocks also provide a set of extension methods that help with ensuring monotonicity. An example of building a custom `IClock` that uses `Environment.TickCount` is shown below.

In [1]:
class SystemClock : IClock
{
    public long Now => Environment.TickCount;
}

Because `TickCount` is a 32-bit integer, the `TickCount` value will overflow after 24.9 days. This will cause our clock to be non-monotonic after running for this duration, which is problematic when calculating relative times, as done by e.g. stopwatches. We can guard against this using the `AssertMonotonic` extension method which will cause an exception to be thrown when the clock goes backwards, rather than producing invalid results.

In [1]:
using System.Threading;

IClock systemClock = new SystemClock();
IClock monotonicSystemClock = systemClock.AssertMonotonic();

Console.WriteLine(monotonicSystemClock.Now);

Thread.Sleep(1000); // If the clock would overflow over here, the following would throw.

Console.WriteLine(monotonicSystemClock.Now);

1072192265


1072193265


Obviously, the issue above could be avoided by using `TickCount64` instead. Nonetheless, facilities to assert or ensure the required invariants of `IClock` instances are provided in this library to safeguard against these types of issues.

## Creative use cases

Clocks do not have to represent time and stopwatches do not have to represent accumulated time difference. Any use case where the result of subtracting `long` values yields a meaningful difference, and adding such differences together yields a meaningful accumulated amount is valid. A good example is the number of bytes allocated on a thread, as shown below.

In [1]:
class MemoryClock : IClock
{
    public long Now => GC.GetAllocatedBytesForCurrentThread();
}

var mem = new MemoryClock();
Console.WriteLine(mem.Now);

1691144


Given this definition of a "clock", we can build "stopwatches" to measure the amount of bytes allocated while the stopwatch is running.

In [1]:
var factory = StopwatchFactory.FromClock(mem);

var swAll = factory.Create();
var sw1 = factory.Create();
var sw2 = factory.Create();
var swPrint = factory.Create();

Now, let's measure the amount of memory allocated by a few operations, using the different stopwatches.

In [1]:
using System.Runtime.CompilerServices;

// Some functions to measure allocations for.
[MethodImpl(MethodImplOptions.NoInlining | MethodImplOptions.NoOptimization)]
void F() => Enumerable.Range(0, 10).Select(x => x.ToString()).ToArray();

[MethodImpl(MethodImplOptions.NoInlining | MethodImplOptions.NoOptimization)]
void G() => string.Join(", ", new[] { "bar", "foo" });

swAll.Start();
{
    sw1.Start();
    {
        F();
    }
    sw1.Stop();

    swPrint.Start();
    {
        Console.WriteLine($"Bytes allocated by F() = {sw1.ElapsedTicks}");
    }
    swPrint.Stop();

    sw2.Start();
    {
        G();
    }
    sw2.Stop();

    swPrint.Start();
    {
        Console.WriteLine($"Bytes allocated by G() = {sw2.ElapsedTicks}");
    }
    swPrint.Stop();
}
swAll.Stop();

Console.WriteLine($"Bytes allocated to print = {swPrint.ElapsedTicks}");
Console.WriteLine($"Total bytes allocated = {swAll.ElapsedTicks}");

Bytes allocated by F() = 640


Bytes allocated by G() = 144


Bytes allocated to print = 8488


Total bytes allocated = 9272


Note that the sum of all allocations measured by `sw1`, `sw2`, and `swPrint` corresponds to the allocations reported by `swAll`.

Using custom stopwatch types, one can also build composite stopwatches that combine multiple underlying stopwatches. For example, an allocation rate could be obtained by dividing bytes allocates by time elapsed. This library does not provide an algebra on top of the `IStopwatch[Factory]` family of interfaces, but it'd be feasible to build using the core abstractions.